Data Pull from https://github.com/slieb74/NBA-Shot-Analysis/blob/master/nba_shots_scraper.py

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import seaborn as sns
import time
import urllib.request
# from court import court_shapes

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)

In [ ]:
# Pull Player IDs from: 
# Original Script from: https://github.com/savvastj/nbashots/blob/master/nbashots/api.py
headers = requests.utils.default_headers()
headers['user-agent'] = ('Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 '
                         '(KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1')
headers['Connection'] = 'closed'

custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9'
}

In [9]:
url = ('https://stats.nba.com/stats/commonallplayers?'
       'IsOnlyCurrentSeason=0&LeagueID=00&Season=2019-20')
response = requests.get(url, headers=headers, stream=True).json()

colnames = response['resultSets'][0]['headers']
players = response['resultSets'][0]['rowSet']

df = pd.DataFrame(players, columns=colnames)

# Lakers only
df = df.loc[df['TEAM_ABBREVIATION']=='LAL',:].reset_index(drop=True)
df

KeyboardInterrupt: 

In [11]:
# https://github.com/slieb74/NBA-Shot-Analysis/blob/master/nba_shots_scraper.py

#year in yyyy-yy format (i.e. '2017-18')
def get_all_players_shot_data(player_ids, year='2019-20'):
    sc_url_1 = 'https://stats.nba.com/stats/shotchartdetail?AheadBehind=&CFID=33&CFPARAMS='
    sc_url_2 = ('&ClutchTime=&Conference=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&'
                'Division=&EndPeriod=10&EndRange=28800&GROUP_ID=&GameEventID=&GameID=&'
                'GameSegment=&GroupID=&GroupMode=&GroupQuantity=5&LastNGames=0&LeagueID=00'
                '&Location=&Month=0&OnOff=&OpponentTeamID=0&Outcome=&PORound=0&Period=0&PlayerID=')
    sc_url_3 = ('&PlayerID1=&PlayerID2=&PlayerID3=&PlayerID4=&PlayerID5=&PlayerPosition=&'
                'PointDiff=&Position=&RangeType=0&RookieYear=&Season=')
    sc_url_4 = ('&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StartPeriod=1&'
                'StartRange=0&StarterBench=&TeamID=0&VsConference=&VsDivision=&VsPlayerID1=&'
                'VsPlayerID2=&VsPlayerID3=&VsPlayerID4=&VsPlayerID5=&VsTeamID=')
    
    all_shots = []
    for player_id in player_ids:
        full_url = sc_url_1 + str(year) + sc_url_2 + str(player_id) + sc_url_3 + str(year) + sc_url_4
        shots = requests.get(full_url, headers=headers).json()
        all_shots.append(shots)
    return all_shots

In [13]:
players_list = df['PERSON_ID'].values.tolist()
all_shots = get_all_players_shot_data(players_list)

KeyboardInterrupt: 

In [ ]:
shots = []
colnames = all_shots[0]['resultSets'][0]['headers']

for k, v in enumerate(players_list):
    for s in all_shots[k]['resultSets'][0]['rowSet']:
        shots.append(s)
        
shot_df = pd.DataFrame(shots, columns=colnames)

In [ ]:
shot_df.head()

In [ ]:
sns.scatterplot(x='LOC_X', y='LOC_Y', hue='SHOT_MADE_FLAG', data=shot_df)

In [ ]:
shot_df.GAME_DATE.nunique()

In [ ]:
shot_df.shape

In [ ]:
# shot_df.to_csv('shot_data.csv')